# Delivery Layout - ansible.cfg
#### ([exercise directory](/tree/notebooks/exercise-00))

In [ ]:
cd /notebooks/exercise-00


When you deliver something you'll probably have a layout:

  - a static or dynamic inventory of all the nodes to manage
  - ssh keys to use
  - users and secrets to connect to the hosts
  - whether to do privilege escalation (eg. sudo, ...) before running tasks
  - if nodes should be accessed via a bastion host, docker, ...
  
Put those information, together with a brief description of the playbook usage (eg. 2/3 lines) into `ansible.cfg`

[![](https://mermaid.ink/img/pako:eNplUz2TmzAQ_SuaLVIBARk4m-KqpEyVLuLmRkYLZg4kRgImjs__PZLAn9eIt0-r3fdW4gSVEggFNJoPB1LKUs586kbmV1LzouZhp6qPt1J-4NEwYw7EgXXLQrtjpv1ynkvT7juMqrphd3hNrtsOiU1X4wH12ui9UrJum1LyaTxcA5TCSbnWbeWMclT6yK7ovub3NWg6tUenJ2GDVmJlDeoZtaMpG9GMX-gN84qiKHreetbRaDUN7zPXht3gRYnqBOpHLc5tMy9qPFwU3CrfDykMXz811qhRVmg-b6adIRJGIbGlnIsV0-ttrXNzFYgnHsbpaXdnXxs-deyVmDo05BvRyn6ZXx_svUEAPeqet8I-mlMpCSnBDq_HEgoLBda-P5TybFOnQfARf4rW2oBi1BMGYJWp30dZXeIl50fL7Yh7sK06Y9mByz9K9ZckG0Jxgr9QJGkapXGWJXFKN9tdvAvgCAXdZVGe7fINzXNKt8nLOYB__nwc5UlG4yzJty80jfM4CQC9oF_Ly_c_wPk_d6YQxA?type=png)](https://mermaid.live/edit#pako:eNplUz2TmzAQ_SuaLVIBARk4m-KqpEyVLuLmRkYLZg4kRgImjs__PZLAn9eIt0-r3fdW4gSVEggFNJoPB1LKUs586kbmV1LzouZhp6qPt1J-4NEwYw7EgXXLQrtjpv1ynkvT7juMqrphd3hNrtsOiU1X4wH12ui9UrJum1LyaTxcA5TCSbnWbeWMclT6yK7ovub3NWg6tUenJ2GDVmJlDeoZtaMpG9GMX-gN84qiKHreetbRaDUN7zPXht3gRYnqBOpHLc5tMy9qPFwU3CrfDykMXz811qhRVmg-b6adIRJGIbGlnIsV0-ttrXNzFYgnHsbpaXdnXxs-deyVmDo05BvRyn6ZXx_svUEAPeqet8I-mlMpCSnBDq_HEgoLBda-P5TybFOnQfARf4rW2oBi1BMGYJWp30dZXeIl50fL7Yh7sK06Y9mByz9K9ZckG0Jxgr9QJGkapXGWJXFKN9tdvAvgCAXdZVGe7fINzXNKt8nLOYB__nwc5UlG4yzJty80jfM4CQC9oF_Ly_c_wPk_d6YQxA)

# ansible.cfg

When running ansible, the first file read is ansible.cfg, resolved in the following order:

  - `ANSIBLE_CONFIG` (process environment variable)
  - `./ansible.cfg` (in the current directory)
  - `~/ansible.cfg` (in the home directory)
  - `/etc/ansible/ansible.cfg`
  
  
`ansible.cfg` is divided in stanzas

```
# defaults, ends with "s". Without "s" it won't work :D
[defaults]
...

[ssh_connection]
...

```

Always check [ansible source code](https://raw.github.com/ansible/ansible/devel/examples/ansible.cfg)  to get in touch with new parameters, for example see the [ansible.cfg stable-2.4](https://github.com/ansible/ansible/blob/stable-2.4/examples/ansible.cfg)

We'll create a new `ansible.cfg` for every project!

#### Exercise

We mentioned a couple of ansible.cfg sections: defaults and ssh_connection.
    
Name a couple more.

In [ ]:
# Write here some more ansible.cfg sections.

In [ ]:
# When running ansible, the first file to be read is
!cat ansible.cfg

#### Exercise 

 - ping all hosts without specifying an inventory file
 - comment the "inventory" line out of [ansible.cfg](/edit/notebooks/exercise-00/ansible.cfg)
 - try to ping then again

In [ ]:
# Use this cell for the exercise
!ansible -m ping all

In [ ]:
# Solution
!sed -i 's/^inventory/#inventory/' ansible.cfg
!ansible -m ping all
!sed -i 's/#inventory/inventory/' ansible.cfg

#### Exercise

You can subscript host groups, eg: `all[0]` is the first host in inventory.

  - ping only the first host
  - then the second

In [ ]:
# Use this cell for the exercise

In [ ]:
# Solution
!ansible -m ping all[0]

#### Exercise

Can you find a default ansible.cfg on this machine? 
If not, have a look at it on your local distro.



## Authentication

You can manage machines via `ssh` or `docker`, but what happens via ssh if `PermitRootLogin=no`?

Just use 
```
[privilege_escalation]
become = yes
become_user = root
become_method = sudo  # defaults to sudo
```

#### Exercise

You can specify which ssh key to use: 

  - which parameter allows to set the default ssh identity?
  - find the answer on the official documentation ;)
  



In [ ]:
# Write here the answer!

## Inventory

The inventory contains the infrastructure hosts. Maintaining an inventory helps to:

  - clearly state each host and its functionalities
  - communicate to others all the involved machines
  - describe the infrastructure

Via `ansible.cfg` you can set a default inventory. You could  eg. default to staging  and require `-i production` to run on actual machines.

Ansible supports dynamic inventories (ldap, script, ..) [see inventory chapter](/notebooks/notebooks/05_inventories.ipynb)

## Encrypt secrets

You can use and deliver secrets in your infrastructure using an encrypted file (aka vault).

Decryption password can be typed each time or can be stored in a pin file configured in `ansible.cfg`.

```
# either
ask_vault_pass = True
# or
vault_password_file = /path/to/pin_file

```


REMEMBER: clear your pin file at logout ;) 

## Bastion

A bastion host is the unique management   entrypoint  for an infrastructure.

Ansible *leverages ssh functionalities* to manage resources from your local machine thru a bastion.
With a proper configuration you can run your commands/playbooks **without** continusly moving files to and fro your bastion.

Those includes:
 
  - socks 
  - local and reverse tunnels (ssh -L | -R )

![title](https://cloud.google.com/solutions/images/bastion.png)
    
  

## Recap Exercise

Check the latest ansible.cfg source code and find 2 parameters you consider useful.

Write down their name and functionality


In [ ]:
# Write the solution here